In [27]:
import pickle
import os
from tqdm import tqdm
import cv2
import numpy as np
motion_path="./motions"

按照帧数进行分隔，因为视频是60帧，所以用秒数来进行输入，后续合并于audio进行合并

In [ ]:
startFrame=0
duration=2
FPS=60
endFrame=startFrame+duration*FPS

video transformer的输入是219维向量
分别是216的关节数据加上3的整体旋转角度
216的关节数据是通过每一帧的24个关节xyz坐标经过cv2的rodrigues来生成3*3的矩阵来生成的，最后进行组合即可

motion文件数据
smpl_poses: Sequences of SMPL pose parameters. Array shape is (N, 24, 3).
smpl_scaling: Human body scaling factor. A scalar value for each sequence.
smpl_trans: Motion 3D trajectory. Array shape is (N, 3).

Rodrigues
处理矩阵三维转换时，通常采用旋转矩阵，但是旋转变换其实只有三个自由度，用旋转向量表达时更为简洁。因此，需要实现从旋转向量和旋转矩阵之间的互转换。
旋转向量和旋转矩阵之间可以通过罗德里格斯公式进行转换：

In [78]:
for file in tqdm(os.listdir(motion_path),desc="processing motion"):
    content=open(r'./motions/'+file,'rb')
    data=pickle.load(content)

    frameDataSet=[]
    for frame in range(startFrame,endFrame,1):
        jointDataSet=[]
        for joint in range(0,24,1):

            x=data["smpl_poses"][frame][3*joint]*data["smpl_scaling"]
            y=data["smpl_poses"][frame][3*joint+1]*data["smpl_scaling"]
            z=data["smpl_poses"][frame][3*joint+2]*data["smpl_scaling"]

            pos=np.asarray([x,y,z])
            #print(pos)
            jointDataSet.append(cv2.Rodrigues(pos)[0].flatten())


        transData=np.asarray(data["smpl_trans"][frame])
        jointDataSet=np.asarray(jointDataSet)


        jointDataSet=jointDataSet.flatten()
        #print(jointDataSet.shape)
        np.append(jointDataSet,transData[0])
        np.append(jointDataSet,transData[1])
        np.append(jointDataSet,transData[2])

        frameDataSet.append(jointDataSet)
        #frameDataSet.append(transData)
        #print(frameDataSet)

    frameDataSet=np.asarray(frameDataSet)
    print(frameDataSet.shape)
    break

processing motion:   0%|          | 0/1408 [00:00<?, ?it/s]


(120, 216)
